In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("udf").getOrCreate()

transactions = [
    ('찹쌀탕수육+짜장2', '2021-11-07 13:20:00', 22000, 'KRW'),
    ('등심탕수육+크립새우+짜장면', '2021-10-24 11:19:00', 21500, 'KRW'), 
    ('월남 쌈 2인 세트', '2021-07-25 11:12:40', 42000, 'KRW'), 
    ('콩국수+열무비빔국수', '2021-07-10 08:20:00', 21250, 'KRW'), 
    ('장어소금+고추장구이', '2021-07-01 05:36:00', 68700, 'KRW'), 
    ('족발', '2020-08-19 19:04:00', 32000, 'KRW'),  
]

schema = ["name", "datetime", "price", "currency"]

In [2]:
df = spark.createDataFrame(data=transactions, schema = schema)
df.createOrReplaceTempView("transactions")

In [3]:
spark.sql("select * from transactions").show()

+--------------------------+-------------------+-----+--------+
|                      name|           datetime|price|currency|
+--------------------------+-------------------+-----+--------+
|          찹쌀탕수육+짜장2|2021-11-07 13:20:00|22000|     KRW|
|등심탕수육+크립새우+짜장면|2021-10-24 11:19:00|21500|     KRW|
|          월남 쌈 2인 세트|2021-07-25 11:12:40|42000|     KRW|
|       콩국수+열무비빔국수|2021-07-10 08:20:00|21250|     KRW|
|       장어소금+고추장구이|2021-07-01 05:36:00|68700|     KRW|
|                      족발|2020-08-19 19:04:00|32000|     KRW|
+--------------------------+-------------------+-----+--------+



# UDF
- UDF : 사용자 정의 함수
- 분산처리 환경에서 사용할수있는 함수를 만들어낸다.

In [4]:
from pyspark.sql.types import LongType

# 마스터 노드에서 사용하는 함수...-> 워커에서 작동하지 않는다.
def squared(n):
    return n*n


In [5]:
# 워커에서 함수가 작동될수있도록 udf 로 등록
spark.udf.register("squared", squared, LongType()) # 워커에서 사용할 함수의 이름, ? , 리턴 타입

<function __main__.squared(n)>

In [6]:
spark.sql("select price, squared(price) from transactions").printSchema() # 보통 쿼리에서 많이 사용하게 된다.

root
 |-- price: long (nullable = true)
 |-- squared(price): long (nullable = true)



In [7]:
spark.sql("select price, squared(price) from transactions").show()

+-----+--------------+
|price|squared(price)|
+-----+--------------+
|22000|     484000000|
|21500|     462250000|
|42000|    1764000000|
|21250|     451562500|
|68700|    4719690000|
|32000|    1024000000|
+-----+--------------+



In [8]:
# 가격을 22000 -> 이만 이천원 이렇게 읽고싶으면?
def read_number(n):
    units = ["", "십", "백", "천", "만"]
    nums = '일이삼사오육칠팔구'
    result = []
    i = 0
    while n > 0:
        n, r = divmod(n, 10)
        if r > 0:
            result.append(nums[r-1]+units[i])
        i += 1
    return "".join(reversed(result))

In [9]:
# test
read_number(22000)

'이만이천'

In [10]:
# 스파크 환경에서 쓸수있도록 UDF 로 등록시켜준다.
spark.udf.register("read_number", read_number) # 타입을 지정하지않으면 문자열 타입으로 리턴이 된다.

<function __main__.read_number(n)>

In [11]:
query = """
select price, read_number(price)
from transactions
"""

spark.sql(query).show()

+-----+------------------+
|price|read_number(price)|
+-----+------------------+
|22000|          이만이천|
|21500|      이만일천오백|
|42000|          사만이천|
|21250|  이만일천이백오십|
|68700|      육만팔천칠백|
|32000|          삼만이천|
+-----+------------------+



In [12]:
# 요일을 구해내는 함수 만들기
def get_weekday(date):
    import calendar
    return calendar.day_name[date.weekday()]
spark.udf.register("get_weekday", get_weekday)

<function __main__.get_weekday(date)>

In [15]:
# test
from datetime import datetime
get_weekday(datetime(2022,5,16))

'Monday'

In [16]:
query = """
select datetime, get_weekday(TO_DATE(datetime)) as day_of_week
from transactions
"""

spark.sql(query).show()

+-------------------+-----------+
|           datetime|day_of_week|
+-------------------+-----------+
|2021-11-07 13:20:00|     Sunday|
|2021-10-24 11:19:00|     Sunday|
|2021-07-25 11:12:40|     Sunday|
|2021-07-10 08:20:00|   Saturday|
|2021-07-01 05:36:00|   Thursday|
|2020-08-19 19:04:00|  Wednesday|
+-------------------+-----------+



In [17]:
spark.stop()